In [1]:
import pandas as pd


In [28]:
train = pd.read_csv('train.csv')

In [18]:
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y_bool
0,51,admin.,married,tertiary,no,148,no,no,cellular,15,apr,263,2,-1,0,unknown,0
1,55,blue-collar,married,secondary,no,7160,yes,no,unknown,4,may,315,1,-1,0,unknown,0
2,65,blue-collar,divorced,tertiary,no,2197,yes,no,cellular,12,may,102,2,-1,0,unknown,0
3,25,admin.,married,secondary,no,6658,yes,no,cellular,16,feb,197,1,-1,0,unknown,1
4,36,services,married,secondary,no,1761,yes,no,cellular,19,sep,177,1,-1,0,success,0


In [ ]:
train.corrwith(train['y_bool']).sort_values(ascending=False)

In [29]:
def preprocess(df):
    df = pd.get_dummies(df, columns=['job', 'marital', 'education', 'contact', 'month', 'poutcome'])
    df['housing'] = df['housing'].map({'yes': 1, 'no': 0})
    df['loan'] = df['loan'].map({'yes': 1, 'no': 0})
    df['default'] = df['default'].map({'yes': 1, 'no': 0})
    return df


In [60]:
#undersampling
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(train.drop(['y_bool'], axis=1), train['y_bool'])
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.3, random_state=42)

In [65]:
#oversampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(train.drop(['y_bool'], axis=1), train['y_bool'])
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.3, random_state=42)

In [30]:
train= preprocess(train)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(['y_bool'], axis=1), train['y_bool'], test_size=0.3, random_state=42)

In [66]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150)

In [67]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=150)

In [68]:
preds=rf.predict_proba(X_test)

In [69]:
# find log loss
from sklearn.metrics import log_loss
log_loss(y_test, preds[:,1])

0.4364891244430466

In [70]:
test = pd.read_csv('test.csv')
test = preprocess(test)
ans=rf.predict_proba(test)

In [71]:
ans = pd.DataFrame(ans[:, 1], columns=['y_bool'])
ans.to_csv('submission4.csv', index=False)
